In [8]:
import os
import sys
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [9]:
#face_cascade = cv2.CascadeClassifier('/usr/local/share/OpenCV/haarcascades/haarcascade_frontalface_default.xml')
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [10]:
def help_message():
   print("Usage: [Question_Number] [Input_Video] [Output_Directory]")
   print("[Question Number]")
   print("1 Camshift")
   print("2 Particle Filter")
   print("3 Kalman Filter")
   print("4 Optical Flow")
   print("[Input_Video]")
   print("Path to the input video")
   print("[Output_Directory]")
   print("Output directory")
   print("Example usages:")
   print(sys.argv[0] + " 1 " + "02-1.avi " + "./")

def detect_one_face(im):
    gray=cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, 1.2, 3)
    if len(faces) == 0:
        return (0,0,0,0)
    return faces[0]

def hsv_histogram_for_window(frame, window):
    # set up the ROI for tracking
    c,r,w,h = window
    roi = frame[r:r+h, c:c+w]
    hsv_roi =  cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv_roi, np.array((0., 60.,32.)), np.array((180.,255.,255.)))
    roi_hist = cv2.calcHist([hsv_roi],[0],mask,[180],[0,180])
    cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)
    return roi_hist


def resample(weights):
    n = len(weights)
    indices = []
    C = [0.] + [sum(weights[:i+1]) for i in range(n)]
    u0, j = np.random.random(), 0
    for u in [(u0+i)/n for i in range(n)]:
      while u > C[j]:
          j+=1
      indices.append(j-1)
    return indices

In [11]:
v = cv2.VideoCapture("02-1 (Converted).mov");

In [12]:
def mycamshift(file_name):
    # Open output file
    output_name = "./"+file_name
    output = open(output_name,"w")

    frameCounter = 0
    # read first frame
    ret ,frame = v.read()
    if ret == False:
        return

    # detect face in first frame
    c,r,w,h = detect_one_face(frame)

    # Write track point for first frame
    pt_x, pt_y = c + w / 2, r + h / 2
    output.write("%d,%d,%d\n" % (0, pt_x, pt_y)) # Write as 0,pt_x,pt_y
    frameCounter = frameCounter + 1

    # set the initial tracking window
    track_window = (c, r, w, h)

    # calculate the HSV histogram in the window
    roi_hist = hsv_histogram_for_window(frame, (c, r, w, h)) # this is provided for you

    term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

    while(True):
        ret, frame = v.read() # read another frame
        if ret == False:
            break

        #change colorspace to hsv
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        #calculating backprojection
        dst = cv2.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)

        #meanshift to get new location
        ret, track_window = cv2.CamShift(dst, track_window, term_crit)

        #drawing it on image
        pts = cv2.boxPoints(ret)

        pt_x = (pts[0][0] + pts[2][0]) / 2
        pt_y = (pts[0][1] + pts[2][1]) / 2

        # write the result to the output file
        output.write("%d,%d,%d\n" % (frameCounter, pt_x, pt_y)) # Write as frame_index,pt_x,pt_y
        frameCounter = frameCounter + 1

    output.close()

In [13]:
mycamshift("output_camshift.txt")

AttributeError: 'module' object has no attribute 'boxPoints'